In [12]:
# get the preprocessed data from the preprocess file
from preprocess import *

## Model Building

In [111]:
def unique_n_gram_occurance(unique_n_grams, flat_n_grams, unique_word): 
    word_count_i = 0
    for i in unique_n_grams: 
        if unique_word == i: 
            for j in flat_n_grams: 
                if i == j: 
                   word_count_i +=1
            return word_count_i
    return 0

In [90]:
def unigram_word_count(unique_word): 
    uni_grams = tokenized_sent['uni_grams']
    
    unique_word_tuple = (unique_word,)
    # Flatten the list of lists
    flat_uni_grams = [word for sublist in uni_grams for word in sublist]
    
    # Convert the flattened list to a set to remove duplicates
    unique_words_set_uni_grams = set(flat_uni_grams)
    
    word_count_i = 0
    for i in unique_words_set_uni_grams: 
        if unique_word_tuple == i: 
            for j in flat_uni_grams: 
                if i == j: 
                   word_count_i +=1
            return word_count_i
    
    return 0 

In [ ]:
# backoff
# this function utilizes the 3 functions following it
def probability_backoff():
    return

In [ ]:
def probability_tri():
    return

In [112]:
def probability_bi(word1, word2):
    bi_grams = tokenized_sent['bi_grams']
    
    unique_word_tuple = (word1, word2)
    
    flat_bi_grams = [word for sublist in bi_gram for word in sublist]
    
    # num_flat_bi_grams = len(flat_bi_grams)
    
    unique_words_set_bi_grams = set(flat_bi_grams)
    
    # num_unique_words_set_bi_grams = len(unique_words_set_bi_grams)
    
    unique_bi_grams_count = unique_n_gram_occurance(unique_words_set_bi_grams, flat_bi_grams, unique_word_tuple)
    uni_gram_word = unique_word_tuple[0]
    uni_gram_word_count = unigram_word_count(uni_gram_word)
    bi_gram_probability = unique_bi_grams_count/uni_gram_word_count
    
    
    return bi_gram_probability

In [121]:
def probability_uni(unique_word):
    uni_grams = tokenized_sent['uni_grams']
    
    unique_word_tuple = (unique_word,)
    # Flatten the list of lists
    flat_uni_grams = [word for sublist in uni_grams for word in sublist]
    
    num_flat_uni_grams = len(flat_uni_grams)
    
    # Convert the flattened list to a set to remove duplicates
    unique_words_set_uni_grams = set(flat_uni_grams)
    
    # Count the number of unique words
    num_unique_words = len(unique_words_set_uni_grams)
    
    #Create dictionary that store each unique word occurance
    unique_uni_grams_count = unique_n_gram_occurance(unique_words_set_uni_grams, flat_uni_grams, unique_word_tuple)
    uni_gram_probability = unique_uni_grams_count/num_flat_uni_grams

    
    return uni_gram_probability
     
    # return unique_word_prob_dict

In [122]:
testing_word_prob = probability_uni('manually')
testing_word_prob


0.00012295585884667404

In [123]:
bi_prop = probability_bi('computation', 'information')
bi_prop

0.125

In [ ]:
# build a model using backoff
def language_model_backoff():
    return

## Model Evaluation

In [ ]:
def perplexity_back_off():
    return

## Text Generation